# Evan Trock Final Data Analysis Project

## Freddie Mac Fannie Mae Data

In [ ]:
import pandas as pd
import statsmodels.api as sm
import numpy as np
from statsmodels.formula.api import ols as sm_ols
from statsmodels.iolib.summary2 import summary_col # nicer tables
# Load the Excel file
df = pd.read_csv('Spring2025_Freddie_Assignment_Data.csv')

df.head()

: 

In [2]:
# Rename relevant columns to standardized names
df_clean = df.rename(columns={
    'Orig. Int Rate': 'Rate',
    'UPB': 'UPB',
    'Loan Term': 'Term',
    'LTV': 'LTV',
    'DTI': 'DTI',
    'Credit Score': 'FICO',
    'Number Borrowers': 'Borrowers',
    'First Time Hombebuyer Flag': 'First',
    'PPM ': 'Penalty' 
})

# Select relevant columns
cols = ['Rate', 'UPB', 'Term', 'LTV', 'DTI', 'FICO', 'Borrowers', 'First', 'Penalty']
df_stats = df_clean[cols].copy()

# Convert indicator variables to binary format
df_stats['First'] = df_stats['First'].apply(lambda x: 1 if str(x).strip().upper() == 'Y' else 0)
df_stats['Penalty'] = df_stats['Penalty'].apply(lambda x: 1 if str(x).strip().upper() == 'Y' else 0)

# Define rounding requirements
rounding = {
    'Rate': 1,'LTV': 1,'DTI': 1, 'Borrowers': 1,'First': 2,'Penalty': 3,'UPB': 0,'Term': 0,'FICO': 0
}

# Compute summary statistics
summary = {}
for col in df_stats.columns:
    stats = {
        'Mean': round(df_stats[col].mean(skipna=True), rounding[col]),
        'Median': round(df_stats[col].median(skipna=True), rounding[col]),
        'Std Dev': round(df_stats[col].std(skipna=True), rounding[col]),
        'Min': round(df_stats[col].min(skipna=True), rounding[col]),
        'Max': round(df_stats[col].max(skipna=True), rounding[col]),
        'Count': int(df_stats[col].count())
    }
    summary[col] = stats

# Convert to DataFrame
summary_df = pd.DataFrame(summary).T

# Display the result
print(summary_df)


                 Mean    Median     Std Dev      Min        Max    Count
Rate            5.200       5.0       1.400      1.8       12.8  75750.0
UPB        207109.000  178000.0  125261.000  10000.0  1564000.0  75750.0
Term          315.000     360.0      76.000     60.0      480.0  75750.0
LTV            71.800      75.0      19.000      6.0      386.0  75750.0
DTI           107.100      36.0     255.400      1.0      999.0  75750.0
FICO          758.000     751.0     401.000    300.0     9999.0  75750.0
Borrowers       1.600       2.0       1.400      1.0       99.0  75750.0
First           0.140       0.0       0.350      0.0        1.0  75750.0
Penalty         0.001       0.0       0.036      0.0        1.0  75750.0


In [3]:
# Select the relevant columns
cols = ['Rate', 'UPB', 'Term', 'LTV', 'DTI', 'FICO', 'Borrowers', 'First', 'Penalty']
df_stats = df_clean[cols].copy()

# Convert 'First' and 'Penalty' to binary indicators
df_stats['First'] = df_stats['First'].apply(lambda x: 1 if str(x).strip().upper() == 'Y' else 0)
df_stats['Penalty'] = df_stats['Penalty'].apply(lambda x: 1 if str(x).strip().upper() == 'Y' else 0)

# Calculate the Pearson correlation matrix
correlation_matrix = df_stats.corr(method='pearson')

# Round to 2 decimal places
correlation_matrix = correlation_matrix.round(2)

# Display the result
print(correlation_matrix)


           Rate   UPB  Term   LTV   DTI  FICO  Borrowers  First  Penalty
Rate       1.00 -0.25  0.19  0.05 -0.10  0.02       0.03   0.03     0.04
UPB       -0.25  1.00  0.22  0.14 -0.08 -0.01       0.03   0.09    -0.03
Term       0.19  0.22  1.00  0.28 -0.07 -0.01      -0.03   0.18    -0.04
LTV        0.05  0.14  0.28  1.00  0.10 -0.02      -0.02   0.26    -0.02
DTI       -0.10 -0.08 -0.07  0.10  1.00 -0.02       0.02  -0.11    -0.01
FICO       0.02 -0.01 -0.01 -0.02 -0.02  1.00      -0.00   0.00     0.01
Borrowers  0.03  0.03 -0.03 -0.02  0.02 -0.00       1.00  -0.04    -0.00
First      0.03  0.09  0.18  0.26 -0.11  0.00      -0.04   1.00    -0.01
Penalty    0.04 -0.03 -0.04 -0.02 -0.01  0.01      -0.00  -0.01     1.00


In [4]:

# Create new variables
df['Purchase'] = df['Loan Purpose'].apply(lambda x: 1 if str(x).strip().upper() == 'PURCHASE' else 0)
df['< 680'] = df['Credit Score'].apply(lambda x: 1 if x < 680 else 0)
df['Single'] = df['Number Borrowers'].apply(lambda x: 1 if x == 1 else 0)

# Select relevant columns
cols = ['oyear', 'Orig. Int Rate', 'Purchase', 'UPB', 'LTV', 'DTI', 'Credit Score', '< 680', 'Single']

# Group by 'Origination Year' and calculate averages
averages_by_year = df[cols].groupby('oyear').mean()

# Add a totals row for overall averages
totals_row = pd.DataFrame(averages_by_year.mean(axis=0)).T
totals_row.index = ['Overall Averages']
averages_with_totals = pd.concat([averages_by_year, totals_row])

# Display the result
averages_with_totals


,Orig. Int Rate,Purchase,UPB,LTV,DTI,Credit Score,< 680,Single
1999,7.334121,0.0,115212.666667,71.376667,75.288000,807.119000,0.261333,0.346333
2000,8.129990,0.0,122996.666667,72.633667,57.186333,863.865000,0.260667,0.394333
2001,6.858851,0.0,138176.333333,70.458000,53.933333,772.529667,0.232667,0.339667
2002,6.396268,0.0,143826.333333,68.317333,52.288000,807.296000,0.211667,0.342333
2003,5.526528,0.0,141283.000000,66.306333,48.020667,732.952667,0.180333,0.328000
2004,5.666535,0.0,157035.333333,68.748667,52.189667,731.975333,0.227000,0.387000
2005,5.810023,0.0,170089.000000,69.457667,61.900667,728.803667,0.234000,0.422667
2006,6.405285,0.0,179737.333333,70.087333,57.525667,722.551333,0.243667,0.446333
2007,6.368495,0.0,181224.000000,71.390667,56.608333,729.514667,0.222000,0.460333
2008,6.058055,0.0,202259.333333,69.995000,54.957333,747.116333,0.143000,0.462333


In [5]:
# Group by 'Seller' and calculate the frequency
seller_stats = df['Seller Name'].value_counts().reset_index()
seller_stats.columns = ['Seller', 'Frequency']

# Calculate the percent of total and cumulative percent
total_loans = seller_stats['Frequency'].sum()
seller_stats['Percent'] = (seller_stats['Frequency'] / total_loans * 100).round(2)
seller_stats['Cumulative'] = seller_stats['Percent'].cumsum().round(2)

# Add a numbering column
seller_stats['Number'] = range(1, len(seller_stats) + 1)

# Reorder columns
seller_stats = seller_stats[['Number', 'Seller', 'Frequency', 'Percent', 'Cumulative']]

# Sort by frequency in descending order
seller_stats = seller_stats.sort_values(by='Frequency', ascending=False)

# Display the result
print(seller_stats)

     Number                           Seller  Frequency  Percent  Cumulative
0         1                    Other sellers      20085    26.51       26.51
1         2           WELLS FARGO BANK, N.A.      10167    13.42       39.93
2         3  WELLS FARGO HOME MORTGAGE, INC.       3659     4.83       44.76
3         4            BANK OF AMERICA, N.A.       3554     4.69       49.45
4         5                   U.S. BANK N.A.       3160     4.17       53.62
..      ...                              ...        ...      ...         ...
104     105           PROSPECT MORTGAGE, LLC          8     0.01       99.93
105     106                        ALLY BANK          7     0.01       99.94
106     107                 NORTHPOINTE BANK          5     0.01       99.95
107     108     PACIFIC UNION FINANCIAL, LLC          3     0.00       99.95
108     109               PULTE MORTGAGE LLC          3     0.00       99.95

[109 rows x 5 columns]


In [6]:
import pandas as pd
import statsmodels.api as sm

# Rename relevant columns
df_clean = df.rename(columns={
    'Orig. Int Rate': 'Rate','UPB': 'UPB','Loan Term': 'Term', 'LTV': 'LTV', 'DTI': 'DTI', 'Credit Score': 'FICO', 'oyear': 'Year'
})

# Keep only relevant columns
df_clean = df_clean[['Rate', 'UPB', 'Term', 'LTV', 'DTI', 'FICO', 'Year']]

# Filter for years 2013–2024 and drop rows with missing data
df_reg = df_clean[(df_clean['Year'] >= 2013) & (df_clean['Year'] <= 2024)].dropna()

# Convert every column explicitly to float to avoid dtype errors
df_reg = df_reg.astype(float)

# Create year dummies (exclude 2013 as base)
year_dummies = pd.get_dummies(df_reg['Year'].astype(int), prefix='Year')
year_dummies.drop('Year_2013', axis=1, inplace=True, errors='ignore')

# Define X and y
X = pd.concat([df_reg[['UPB', 'Term', 'LTV', 'DTI', 'FICO']], year_dummies], axis=1)
y = df_reg['Rate']


# Ensure everything is float64
X = X.astype(float)
y = y.astype(float)

# Add constant
X = sm.add_constant(X)

# Run regression
model = sm.OLS(y, X)
results = model.fit()

# Print regression table
print(results.summary())


                            OLS Regression Results                            
Dep. Variable:                   Rate   R-squared:                       0.792
Model:                            OLS   Adj. R-squared:                  0.791
Method:                 Least Squares   F-statistic:                     8004.
Date:                Wed, 16 Apr 2025   Prob (F-statistic):               0.00
Time:                        16:30:07   Log-Likelihood:                -27399.
No. Observations:               33750   AIC:                         5.483e+04
Df Residuals:                   33733   BIC:                         5.498e+04
Df Model:                          16                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          2.7672      0.025    112.140      0.0